In [1]:
# import required libraries and declare environment variables
import requests
from sqlalchemy import create_engine, text
import pandas as pd

# This is the url to request
URL = "http://mini-challenge.foris.ai"

In [2]:
# create request session in order to store authentication token
session = requests.Session()


In [3]:
# request login url in order to get the JWT
login_response = session.post(url = f"{URL}/login", data = '{"username": "foris_challenge", "password": "ForisChallenge"}', headers = {"Content-Type": "application/json"})
access_token = login_response.json()["access_token"]

In [4]:
# request the challenge information
authorization_headers={"Authorization": f"Bearer {access_token}"}
challenge_response = session.get(f"{URL}/challenge",headers=authorization_headers)
print(challenge_response.text)

Welcome to the Foris developer mini-challenge! Follow these steps to complete the task:

1. **Download SQL Dump**

    Choose between a MySQL or a PostgreSQL dump:
    - For a local MySQL (version 8) environment, download the dump from the endpoint /dumps/mysql.
    - For a local PostgreSQL (versions 13-15) environment, download the dump from the endpoint /dumps/psql.
    - For an online PostgreSQL environment, download the SQL script from the endpoint /dumps/onlinepsql.
   
   Note: If you are using Postman, make sure you execute the request with `Send and Download` as shown here https://stackoverflow.com/questions/38975718/how-to-download-excel-xls-file-from-api-in-postman.

2. **Import Database**

    If you chose one of the local SQL environments, import the downloaded SQL dump into your local environment by your preferred method (tested via terminal). Make sure the import process completes successfully.

    If you chose the online PostgreSQL environment, head to https://sqliteonl

In [6]:
# get dump of the database in mysql
dump_response = session.get(f"{URL}/dumps/mysql",headers = authorization_headers)
print(f"status code: {dump_response.status_code}") # if status code is 200 everything is ok
# Save sql file
with open('./recruitment_test.sql','w') as file:
    file.write(dump_response.text)

status code: 200


Here you have to create the database using mysql. The step by step guide of how I made it, along with some external references are provided bellow:

1) [Install mysql]((https://learn.microsoft.com/en-us/windows/wsl/install)) in the WSL ([Windows Subsystem for Linux](https://learn.microsoft.com/en-us/windows/wsl/install))

2) In the WSL terminal initialize and configure mysql
3) Using mysql server create the database with the following command:
´´´
sudo mysql -h localhost recruitment_test < recruitment_test.sql
´´´
4) I had some issues inserting the enrollment table. So I have corrected the line 44 using insert ignore instead of simple insert statement

And the Database is ready to be queried


In [7]:
# Define the database connection parameters (or better load them from a .env file)
DB_HOST = "localhost"
DB_USER = "foris"
DB_PASSWORD = "password"
DB_NAME = "recruitment_test"
# Create a SQLAlchemy engine with temporary database
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

engine = create_engine(connection_string)
connection = engine.connect()

* `NUMBER_OF_GROUPS`: Is the number of groups present in the table `group` of the given dump. By giving this you prove that you successfully inspect your database.

In [8]:
# calculate the number of groups
inspect_group_query = text("""SELECT * FROM `group`;""")
result = pd.read_sql(inspect_group_query, con=engine)
number_of_groups = result.shape[0]

In [9]:
number_of_groups

168

* `YOUR_ANSWER`: Use an SQL query to find the answer to the following question: "How many students who enrolled in at least 5 groups between the start of February 5 and the end of February 7, enrolled in at least one subject with a code ending in `1`?"

   **Warning!** The value `2024-02-07` represents the start of February 7, not its end.

In [10]:
# calculate the answer
answer_query = text("""
SELECT student_id FROM
(
SELECT e.student_id AS student_id, COUNT(DISTINCT e.group_id) AS enrolled_groups
FROM enrollment e
JOIN `group` g ON e.group_id = g.id
JOIN subject sj ON g.subject_id = sj.id
WHERE e.enrollment_hour >= '2024-02-05' AND e.enrollment_hour < '2024-02-08'
GROUP BY student_id
HAVING COUNT(DISTINCT e.group_id) >= 5
) AS fwetfwe
WHERE student_id IN
(
SELECT e.student_id AS student_id
FROM enrollment e
JOIN `group` g ON e.group_id = g.id
JOIN subject sj ON g.subject_id = sj.id
WHERE sj.code LIKE '%1'
)
""")

result = pd.read_sql_query(answer_query, con=engine)
your_answer = result.size
connection.close()

In [11]:
your_answer

28

In [12]:
# post validation
header={**authorization_headers,**{"Content-Type": "application/json"}}
body = str(f'{{"NUMBER_OF_GROUPS" : {number_of_groups},"YOUR_ANSWER" :{your_answer}}}')


In [13]:
body

'{"NUMBER_OF_GROUPS" : 168,"YOUR_ANSWER" :28}'

In [14]:

response = session.post(url = f"{URL}/validate", data = '{"NUMBER_OF_GROUPS":"168","YOUR_ANSWER":"28"}', headers = header)

In [15]:
response.text


'{"error":"The number of groups is not correct."}\n'

He verificado que el número de grupos es correcto, no sé qué pasa 🥲

In [16]:
session.close()